# DnD Monster Data Wrangling


## Importation

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import re
from bioinfokit.analys import stat
from src.features.build_features import clean_list
from word2number import w2n

%matplotlib inline

monster_df = pd.read_csv('../data/raw/Monster_Data_RAW.csv')

monster_df.head()

,Unnamed: 0,Monster Name,Size,Type,Alignment,Traits,Damage Resistances,Monster Tags:,Mythic Actions,Reactions,...,Proficiency Bonus,STR,DEX,CON,INT,WIS,CHA,Actions,Legendary Actions,Environment:
0,0,Adult Green Dragon,Huge,['dragon'],lawful evil,['Amphibious. The dragon can breathe air and w...,NaN,NaN,NaN,NaN,...,5,23,12,21,18,15,17,['Multiattack. The dragon can use its Frightfu...,"[""The dragon can take 3 legendary actions, cho...",['Forest']
1,1,Adult Silver Dragon,Huge,['dragon'],lawful good,['Legendary Resistance (3/Day). If the dragon ...,NaN,NaN,NaN,NaN,...,5,27,10,25,16,13,21,['Multiattack. The dragon can use its Frightfu...,"[""The dragon can take 3 legendary actions, cho...","['Mountain', 'Urban']"
2,2,Adult White Dragon,Huge,['dragon'],chaotic evil,"[""Ice Walk. The dragon can move across and cli...",NaN,NaN,NaN,NaN,...,5,22,10,22,8,12,12,['Multiattack. The dragon can use its Frightfu...,"[""The dragon can take 3 legendary actions, cho...",['Arctic']
3,3,Air Elemental,Large,['elemental'],neutral,"[""Air Form. The elemental can enter a hostile ...","Lightning, Thunder; Bludgeoning, Piercing, and...",NaN,NaN,NaN,...,3,14,20,14,6,10,6,['Multiattack. The elemental makes two slam at...,NaN,"['Desert', 'Mountain']"
4,4,Ape,Medium,['beast'],unaligned,[nan],NaN,['Misc Creature'],NaN,NaN,...,2,16,14,14,6,12,7,['Multiattack. The ape makes two fist attacks....,NaN,['Forest']


## Performing the Basic Cleanup
Removing columns we won't use, cleanining up feature titles, and checking out our dataset for datatypes and issues.

We know that Mythic Actions weren't introducted until a later version, so no monsters contain this category.

Unnamed:0 is useless, Monster Tags: is the same as Monster type (with a bit more specificity, which we don't need.)

Skills, Source, Launguages, and Senses are all unecessary for our MVP.

In [2]:
monster_df.drop(columns = {'Unnamed: 0', 'Mythic Actions', 'Monster Tags:', "Skills", 'Source', 'Languages', "Senses"}, inplace = True, axis = 1)
monster_df.rename(columns = {"Environment:":"Environment"}, inplace=True)

In [3]:
print(monster_df.columns)
print(monster_df.describe())

Index(['Monster Name', 'Size', 'Type', 'Alignment', 'Traits',
       'Damage Resistances', 'Reactions', 'Armor Class', 'Hit Points', 'Speed',
       'Saving Throws', 'Damage Vulnerabilities', 'Damage Immunities',
       'Condition Immunities', 'Challenge', 'Proficiency Bonus', 'STR', 'DEX',
       'CON', 'INT', 'WIS', 'CHA', 'Actions', 'Legendary Actions',
       'Environment'],
      dtype='object')
       Armor Class  Hit Points  Proficiency Bonus         STR         DEX  \
count   348.000000  348.000000         348.000000  348.000000  348.000000   
mean     13.985632   78.399425           2.712644   14.951149   12.706897   
std       3.155403   96.670352           1.296486    6.705018    3.078279   
min       5.000000    1.000000           2.000000    1.000000    1.000000   
25%      12.000000   17.000000           2.000000   11.000000   10.000000   
50%      13.000000   45.000000           2.000000   16.000000   13.000000   
75%      16.000000  110.000000           3.000000   19.00

## Challenge Rating (CR)
Currently Challenge Rating is a string, we want it to be an integer so we can use it. We need to remove the experience string attached and convert.

In [4]:
#split the string and only take the first part (Challenge Rating)
monster_df["Challenge"] = monster_df["Challenge"].str.split().str[0]

#turn fraction strings into floats
for indx, challenge in enumerate(monster_df["Challenge"]):
    if "/" in challenge:
       monster_df.loc[indx,'Challenge'] = pd.eval(challenge)
    else:
        monster_df.loc[indx,'Challenge'] = pd.to_numeric(challenge)

monster_df["Challenge"] = pd.to_numeric(monster_df["Challenge"])

## Monster Type
There are a large number of monster sub-types, this is unecessary for our analysis. we want to consolidate. 

In [5]:
for indx,Type in enumerate(monster_df['Type']):
    monster_df.loc[indx,"Type"] = Type.split(",")[0].strip("[']")

## Missing Values

All features with missing values make sense esxcept Actions, which I would have assumed every creature has an action, that may not be the case however. For the others, they are all optional features that lower level monsters won't have.

The NAs may still cause issues down the road, so I will replace them all with a string text like "NA"

In [6]:
monster_df.isna().any()

monster_df.fillna("['NA']",inplace=True)

## List Values
We current have features where the values are varying list of items. For example, a monster may be found in more than one environment such as [mountain, coastal, underdark]. Unfortuntaly, they are reading as strings right now so we will need to convert them to lists for easy of use.

In [7]:
#All lists columns are actually strings!
for i,j in enumerate(monster_df["Environment"]):
   print("list",i,"is",type(j))

list 0 is <class 'str'>
list 1 is <class 'str'>
list 2 is <class 'str'>
list 3 is <class 'str'>
list 4 is <class 'str'>
list 5 is <class 'str'>
list 6 is <class 'str'>
list 7 is <class 'str'>
list 8 is <class 'str'>
list 9 is <class 'str'>
list 10 is <class 'str'>
list 11 is <class 'str'>
list 12 is <class 'str'>
list 13 is <class 'str'>
list 14 is <class 'str'>
list 15 is <class 'str'>
list 16 is <class 'str'>
list 17 is <class 'str'>
list 18 is <class 'str'>
list 19 is <class 'str'>
list 20 is <class 'str'>
list 21 is <class 'str'>
list 22 is <class 'str'>
list 23 is <class 'str'>
list 24 is <class 'str'>
list 25 is <class 'str'>
list 26 is <class 'str'>
list 27 is <class 'str'>
list 28 is <class 'str'>
list 29 is <class 'str'>
list 30 is <class 'str'>
list 31 is <class 'str'>
list 32 is <class 'str'>
list 33 is <class 'str'>
list 34 is <class 'str'>
list 35 is <class 'str'>
list 36 is <class 'str'>
list 37 is <class 'str'>
list 38 is <class 'str'>
list 39 is <class 'str'>
list 40 is

### Attack, Spell Attack, Save DC
Actually, there is some information within these strings we can pull out easily with regex search, match, findall. Let's do that before converting into lists

In [8]:
# Create new columns for features
monster_df = monster_df.assign(Attack_Bonus= '', Spell_Bonus = '', Spell_Save_DC = '')

#Attack Bonus
for indx, action in enumerate(monster_df['Actions']):
    try:
        found = re.search("\+(.+?) to hit", action).group(0)
        monster_df.loc[indx,'Attack_Bonus'] = int(found.split()[0].lstrip('+'))
    except:
        monster_df.loc[indx,'Attack_Bonus'] = 0

#Spell Attack Bonus

for indx, trait in enumerate(monster_df['Traits']):
    try:
        found = re.search("\+(.+?) to hit", trait).group(0)
        monster_df.loc[indx,'Spell_Bonus'] = int(found.split()[0].lstrip('+'))
    except:
        monster_df.loc[indx,'Spell_Bonus'] = 0

#Spell Save DC

for indx, trait in enumerate(monster_df['Traits']):
    try:
        found = re.search("spell save DC [0-9]+", trait).group(0)
        monster_df.loc[indx,'Spell_Save_DC'] = int(found.split()[-1])
    except:
        monster_df.loc[indx,'Spell_Save_DC'] = 0

## Saving Throw Expansion
We want to be able to evaluate Saving Throw Numbers just like we due stats. Some monsters have bonuses to certain saving throws, which we will input first. Then we will use the stats to fill in the rest of the saving throws. Stat numbers have a base relationship to saving throw where every 2 stat increases is +1 into Saving throw. Example, 10 in Strength is a +0 in Str Saving Throw, but a 12 in Strength is a +1, and finally a 20 in Strength is a +5 in Strength saving throw.

In [9]:
monster_df["Saving Throws"] = monster_df['Saving Throws'].apply(clean_list)

#turn saving throw feature values into lists using literal_eval
monster_df["Saving Throws"] = monster_df["Saving Throws"].apply(ast.literal_eval)

#Saving Throw Exapanded features
saving_throw_df = pd.DataFrame(columns={"STR_SV","DEX_SV","CON_SV","INT_SV","WIS_SV","CHA_SV"})

for indx, saving_throw in enumerate(monster_df['Saving Throws']): 
    for string in saving_throw:
        if "DEX" in string:
            saving_throw_df.loc[indx,"DEX_SV"] = int(string.split()[1].lstrip('+'))
        elif "CON" in string:
            saving_throw_df.loc[indx,"CON_SV"] = int(string.split()[1].lstrip('+'))
        elif "STR" in string:
            saving_throw_df.loc[indx,"STR_SV"] = int(string.split()[1].lstrip('+'))
        elif "WIS" in string:
            saving_throw_df.loc[indx,"WIS_SV"] = int(string.split()[1].lstrip('+'))
        elif "INT" in string:
            saving_throw_df.loc[indx,"INT_SV"] = int(string.split()[1].lstrip('+'))
        elif "CHA" in string:
            saving_throw_df.loc[indx,"CHA_SV"] = int(string.split()[1].lstrip('+'))
            
monster_df = pd.concat([monster_df,saving_throw_df], axis=1)

In [10]:
#Using Stats to fill in missing saving throws
stat_modifiers ={('1') : -5, ('2','3') : -4, ('4','5') : -3, ('6','7') : -2, ('8','9') : -1, ('10','11') : 0, ('12','13') : 1, ('14','15') : 2, ('16','17') : 3, ('18','19') : 4, 
('20','21') : 5, ('22','23') : 6, ('24','25') : 7, ('26','27') : 8, ('28','29') : 9, ('30') : 10}

for clms in monster_df.iloc[:,29:34]:
    monster_stat = clms.split('_')[0]
    for indx, value in enumerate(monster_df[clms]):
        if math.isnan(value):
            for stat_num, modifier in stat_modifiers.items():
               if str(monster_df.loc[indx,monster_stat]) in stat_num:
                    monster_df.loc[indx,clms] = modifier
          

In [11]:

#evaluate string and turn into lists
column_lists = ["Environment", "Reactions", "Actions", "Legendary Actions"]
for columns in column_lists:
    monster_df[columns] = monster_df[columns].apply(ast.literal_eval)
            
#"Damage Resistances","Damage Vulnerabilities", "Damage Immunities", have wonky typing due to semicolon
# Traits, Condition immunities, saving throws create type error

#check that they are lists
for i,j in enumerate(monster_df["Environment"]):
   print("list",i,"is",type(j))

#create dummy variables for envinroment, which includes the list for variables
dummies = pd.get_dummies(monster_df['Environment'].explode()).reset_index().groupby(['index']).sum()
monster_df = pd.concat([monster_df,dummies], axis=1)

list 0 is <class 'list'>
list 1 is <class 'list'>
list 2 is <class 'list'>
list 3 is <class 'list'>
list 4 is <class 'list'>
list 5 is <class 'list'>
list 6 is <class 'list'>
list 7 is <class 'list'>
list 8 is <class 'list'>
list 9 is <class 'list'>
list 10 is <class 'list'>
list 11 is <class 'list'>
list 12 is <class 'list'>
list 13 is <class 'list'>
list 14 is <class 'list'>
list 15 is <class 'list'>
list 16 is <class 'list'>
list 17 is <class 'list'>
list 18 is <class 'list'>
list 19 is <class 'list'>
list 20 is <class 'list'>
list 21 is <class 'list'>
list 22 is <class 'list'>
list 23 is <class 'list'>
list 24 is <class 'list'>
list 25 is <class 'list'>
list 26 is <class 'list'>
list 27 is <class 'list'>
list 28 is <class 'list'>
list 29 is <class 'list'>
list 30 is <class 'list'>
list 31 is <class 'list'>
list 32 is <class 'list'>
list 33 is <class 'list'>
list 34 is <class 'list'>
list 35 is <class 'list'>
list 36 is <class 'list'>
list 37 is <class 'list'>
list 38 is <class 'lis

## Actions: Damage
While there is a ton of information in Actions that we may use for word clouds later, the most critical thing for our MVP is trying to pull out the potential damage of the monsters. This is difficult since monsters are so diverse, some have multiattack, which could mean many different things, some have spells, which we don't immediatelly have the damage for, some do secondary damage upon a failed saving throw. So distilling this down into a simple X damage per round will prove difficult. 

First, we can see that regular attacks follow a pattern of 'Hit: XX (XdX + X) """ """ damage.' This is important because we can pull out the average damage and use it for the monster. I'm thinking we may need to make a seperate dataframe to work with this information to start.

In [42]:
monster_actions = monster_df[['Monster Name', 'Actions']]

#Find out max number of attacks is 10
max_attacks = monster_actions['Actions'].explode()
max_attacks.groupby(max_attacks.index).count().max()

#Create 10 columns to work through attacks individually
monster_actions = monster_actions.assign(Attack_1 = "", Attack_2 = "",  Attack_3 = "", Attack_4 = "", Attack_5 = "", Attack_6 = "", Attack_7 = "", Attack_8 = "", Attack_9 = "", Attack_10 = "")

for indx,actions in enumerate(monster_actions['Actions']):
    n = 0
    for action in actions:
        monster_actions.iloc[indx, n+2] = action
        n+=1
        
monster_actions.drop("Actions", inplace=True, axis=1)

In [63]:
#update column to show dictionary of types and number of attacks for multiattacks

def MultiAttackSearch(attack_value, search, replaced, replace, split):
    multiattack = re.search(search, attack_value).group(1)
    multiattack = multiattack.replace(replaced,replace)
    multiattack = re.split(split, multiattack)
    for indx1, item in enumerate(multiattack):
        if item !=" ":
            MA_number = {}
            multiattack[indx1] = item.split()
            value, key = multiattack[indx1][0], multiattack[indx1][1]
            MA_number[key] = w2n.word_to_num(value)
            multiattack[indx1] = MA_number
    return multiattack

for indx, attack in enumerate(monster_actions["Attack_1"]):
        if "Multiattack" in attack:
            if ": " in attack:
                 monster_actions.loc[indx,"Attack_1"] = MultiAttackSearch(attack,"\: (.*?)\.", " with its "," ",'and |,')
            elif ("makes " in attack) and ("1d4" not in attack) and ("either" not in attack) and ("as" not in attack):   
                multiattack = re.search("makes (.*?) ", attack).group(1)
                multiattack = multiattack.replace("makes ","")
                multiattack = w2n.word_to_num(multiattack)
                MA_number = {}
                value, key = multiattack, "Attack"
                MA_number[key] = value
                Monster_list = []
                Monster_list.append(MA_number)
                monster_actions.loc[indx,"Attack_1"] = Monster_list
            elif ("medusa" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{'snake hair':1},{'shortsword':2}]
            elif ("drider" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{'longsword':3}]
            elif ("flameskull" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{'Fire Ray':2}]
            elif ("oni" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{'Glaive':2}]
            elif ("fungus" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{"Rotting Touch": 4}]
            elif ("hydra" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{"Bite": 5}]
            elif ("assassin" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{"Bite": 5}]
            elif ("rakshasa" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{"Bite": 5}]
            elif ("veteran" in attack):
                monster_actions.loc[indx,"Attack_1"] = [{"longsword": 2},{"shortsword":1}]


            #     int(attack.split('.')[1].split("uses")[1].split())
            #     flame1 = re.sub('[^a-zA-Z0-9 \n\.]', '', str(flame[0:2]))
            #     value,key = 2,flame1
            #     MA_number = {}
            #     MA_number[key] = value
            #     print(MA_number)

In [40]:
#replace "Hit:" attacks with just damage and bonus damage such as extra lightning damage
for col in monster_actions.iloc[:,1:11]:
    for indx, attack in enumerate(monster_actions[col]):
        try:
            if "plus" in attack:
                ext_damage = int(re.search("plus (.+?) ", attack).group(1))
                if "Hit:" in attack:
                    prim_damage = int(re.search("Hit: (.+?) ", attack).group(1))
                elif "taking" and "damage" in attack:
                    prim_damage =  re.search("taking (.+?) ", attack).group(1)
                elif "target takes" and "damage" in attack:
                    prim_damage =  re.search("takes [0-9]+ ", attack).group(0).split()[1]
                elif "if the creature takes damage" or "deals damage" in attack:
                    prim_damage =  0
                elif "creature takes" and "damage" in attack:
                    prim_damage =  re.search("takes [0-9]+ ", attack).group(0).split()[1]
                monster_actions.loc[indx,col] = prim_damage + ext_damage
            else:
                if "Hit:" in attack:
                    monster_actions.loc[indx,col] = re.search("Hit: (.+?) ", attack).group(1)
                elif "taking" and "damage" in attack:
                    monster_actions.loc[indx,col] = re.search("taking (.+?) ", attack).group(1)
                elif "target takes" and "damage" in attack:
                    monster_actions.loc[indx,col] = re.search("takes [0-9]+ ", attack).group(0).split()[1]
                elif "if the creature takes damage" or "deals damage" in attack:
                    monster_actions.loc[indx,col] = 0
                elif "creature takes" and "damage" in attack:
                    monster_actions.loc[indx,col] = re.search("takes [0-9]+ ", attack).group(0).split()[1]
        except:
            continue

In [66]:
monster_actions

,Monster Name,Attack_1,Attack_2,Attack_3,Attack_4,Attack_5,Attack_6,Attack_7,Attack_8,Attack_9,Attack_10
0,Adult Green Dragon,"[{'bite': 1}, {'claws': 2}]","Bite. Melee Weapon Attack: +11 to hit, reach 1...","Claw. Melee Weapon Attack: +11 to hit, reach 5...","Tail. Melee Weapon Attack: +11 to hit, reach 1...",Frightful Presence. Each creature of the drago...,Poison Breath (Recharge 5–6). The dragon exhal...,,,,
1,Adult Silver Dragon,"[{'bite': 1}, {'claws': 2}]","Bite. Melee Weapon Attack: +13 to hit, reach 1...","Claw. Melee Weapon Attack: +13 to hit, reach 5...","Tail. Melee Weapon Attack: +13 to hit, reach 1...",Frightful Presence. Each creature of the drago...,Breath Weapons (Recharge 5–6). The dragon uses...,Cold Breath. The dragon exhales an icy blast i...,Paralyzing Breath. The dragon exhales paralyzi...,Change Shape. The dragon magically polymorphs ...,"In a new form, the dragon retains its alignmen..."
2,Adult White Dragon,"[{'bite': 1}, {'claws': 2}]","Bite. Melee Weapon Attack: +11 to hit, reach 1...","Claw. Melee Weapon Attack: +11 to hit, reach 5...","Tail. Melee Weapon Attack: +11 to hit, reach 1...",Frightful Presence. Each creature of the drago...,Cold Breath (Recharge 5–6). The dragon exhales...,,,,
3,Air Elemental,[{'Attack': 2}],"Slam. Melee Weapon Attack: +8 to hit, reach 5 ...",Whirlwind (Recharge 4–6). Each creature in the...,"If the saving throw is successful, the target ...",,,,,,
4,Ape,[{'Attack': 2}],"Fist. Melee Weapon Attack: +5 to hit, reach 5 ...","Rock. Ranged Weapon Attack: +5 to hit, range 2...",,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
343,Wyvern,"[{'bite': 1}, {'stinger': 1}]","Bite. Melee Weapon Attack: +7 to hit, reach 10...","Claws. Melee Weapon Attack: +7 to hit, reach 5...","Stinger. Melee Weapon Attack: +7 to hit, reach...",,,,,,
344,Zombie,"Slam. Melee Weapon Attack: +3 to hit, reach 5 ...",,,,,,,,,
345,Commoner,"Club. Melee Weapon Attack: +2 to hit, reach 5 ...",,,,,,,,,
346,Giant Owl,"Talons. Melee Weapon Attack: +3 to hit, reach ...",,,,,,,,,


In [ ]:
#find the phrase that dictates multiattacks
for indx, attack in enumerate(monster_actions["Attack_1"]):
    try:
        if "Multiattack" in attack:
           # print(re.search("makes (.+?) ", attack).group(1))

        continue



In [30]:
#max number of unique attacks are 4 from the tarrasque
monster_actions['Attack_1']

0      0
1      0
2      0
3      0
4      0
      ..
343    0
344    4
345    2
346    8
347    0
Name: Attack_1, Length: 348, dtype: object

In [ ]:
monster_multiattack = monster_actions[monster_actions['Attack_1'].str.contains("Multiattack")==True]

In [ ]:
monster_multiattack[monster_multiattack['Attack_1'].str.contains(": ")].describe()

,Monster Name,Attack_1,Attack_2,Attack_3,Attack_4,Attack_5,Attack_6,Attack_7,Attack_8,Attack_9,Attack_10
count,79,79,79,79,79,79,79,79,79,79,79
unique,79,49,73,58,41,25,23,14,12,2,2
top,Adult Green Dragon,Multiattack. The dragon can use its Frightful ...,"Bite. Melee Weapon Attack: +6 to hit, reach 5 ...","Claw. Melee Weapon Attack: +7 to hit, reach 5 ...",,,,,,,
freq,1,20,2,7,23,43,49,66,68,71,71


In [ ]:
#replace "Hit:" attacks with just damage and bonus damage such as extra lightning damage
for col in monster_multiattack.iloc[:,1:11]:
    for indx, attack in enumerate(monster_actions[col]):
        try:
            if "Hit: " in attack:
                ext_damage = int(re.search("plus (.+?) ", attack).group(1))

SyntaxError: expected 'except' or 'finally' block (283256201.py, line 6)

In [32]:
monster_actions[monster_actions['Attack_1'].str.contains("Multiattack")==True].describe()
# 150 monsters have multiattack FML



,Monster Name,Attack_1,Attack_2,Attack_3,Attack_4,Attack_5,Attack_6,Attack_7,Attack_8,Attack_9,Attack_10
count,0,0,0,0,0,0,0,0,0,0,0
unique,0,0,0,0,0,0,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
monster_actions[monster_actions['Attack_2'].str.contains("damage")==True]

,Monster Name,Attack_1,Attack_2,Attack_3,Attack_4,Attack_5,Attack_6,Attack_7,Attack_8,Attack_9,Attack_10
12,Bulette,30,Deadly Leap. If the bulette jumps at least 15 ...,0,0,0,0,0,0,0,0
251,Giant Toad,12,Swallow. The toad makes one bite attack agains...,0,0,0,0,0,0,0,0
302,Giant Frog,4,Swallow. The frog makes one bite attack agains...,0,0,0,0,0,0,0,0
314,Old Croaker,12,Swallow. The toad makes one bite attack agains...,0,0,0,0,0,0,0,0
340,Remorhaz,50,Swallow. The remorhaz makes one bite attack ag...,If the remorhaz takes 30 damage or more on a s...,0,0,0,0,0,0,0


In [ ]:
for col in monster_actions.iloc[:,1:11]:
    for indx, attack in enumerate(monster_actions[col]):
        try:
            if ("target takes" or "creature takes" or "it takes") and "damage" in attack:
                print(re.search("takes [0-9]+ ", attack).group(0).split()[1], monster_actions.loc[indx,"Monster Name"])
        except:
            continue

10 Giant Toad
5 Giant Frog
10 Old Croaker
21 Remorhaz
15 Air Elemental
13 Water Elemental
15 Dust Devil
30 Purple Worm
10 Banshee
50 Kraken
30 Remorhaz
10 Gelatinous Cube
5 Vrock
3 Kraken
21 Behir
44 Androsphinx
30 Behir
56 Tarrasque
60 Tarrasque


In [36]:
monster_actions

,Monster Name,Attack_1,Attack_2,Attack_3,Attack_4,Attack_5,Attack_6,Attack_7,Attack_8,Attack_9,Attack_10
0,Adult Green Dragon,0,24,13,15,0,56,0,0,0,0
1,Adult Silver Dragon,0,19,15,17,0,0,58,0,0,0
2,Adult White Dragon,0,21,13,15,0,54,0,0,0,0
3,Air Elemental,0,14,Whirlwind (Recharge 4–6). Each creature in the...,"If the saving throw is successful, the target ...",0,0,0,0,0,0
4,Ape,0,6,6,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
343,Wyvern,0,11,13,11,0,0,0,0,0,0
344,Zombie,4,0,0,0,0,0,0,0,0,0
345,Commoner,2,0,0,0,0,0,0,0,0,0
346,Giant Owl,8,0,0,0,0,0,0,0,0,0
